In [ ]:
from anaconda.enterprise.server.sdk.contract.dto.client_options import ClientOptions

options_dict: dict = {
    "hostname": "",
    "username": "",
    "password": "",
}

options: ClientOptions = ClientOptions.parse_obj(options_dict)

In [ ]:
from anaconda.enterprise.server.sdk.session.factory import AESessionFactory
from anaconda.enterprise.server.sdk.client import AEClient

session_factory: AESessionFactory = AESessionFactory(options=options)

In [ ]:
client: AEClient = AEClient(session_factory=session_factory)

In [ ]:
from anaconda.enterprise.server.contracts import ProjectUploadResponse

upload_response: ProjectUploadResponse = client.project_upload(
    project_archive_path="../assets/anaconda.enterprise.mlflow.tracking.server.0.5.0.tar.gz",
    name="dev.mlflow.tracking.server",
)

In [ ]:
upload_response

In [ ]:
upload_response.id

In [ ]:
project_revisions = client.project_revisions_get(project_id=upload_response.id)

In [ ]:
project_revisions

In [ ]:
secret_names: list[str] = ["MLFLOW_BACKEND_STORE_URI", "MLFLOW_ARTIFACTS_DESTINATION", "MLFLOW_TRACKING_GC_TTL"]
for name in secret_names:
    try:
        client.secret_delete(key=name)
    except:
        pass

client.secret_put(key="MLFLOW_BACKEND_STORE_URI", value="sqlite:///data/mlflow/dev/store/mydb.sqlite")
client.secret_put(key="MLFLOW_ARTIFACTS_DESTINATION", value="data/mlflow/dev/artifacts")
client.secret_put(key="MLFLOW_TRACKING_GC_TTL", value="30d0h0m0s")

In [ ]:
deploy_params: dict = {
    "project_id": upload_response.id,
    "deployment_name": "dev-mlflow-tracking-server",
    "revision_id": project_revisions[0].id,
    "command": project_revisions[0].commands[0].id,
    "variables": {
        # "MLFLOW_BACKEND_STORE_URI": "sqlite:///data/mlflow/dev/store/mydb.sqlite",
        # "MLFLOW_ARTIFACTS_DESTINATION": "data/mlflow/dev/artifacts",
        # "MLFLOW_TRACKING_GC_TTL": "30d0h0m0s"
    },
    "static_endpoint": "dev-mlflow-tracking-server",
}
deploy_params

In [ ]:
from anaconda.enterprise.server.contracts import ProjectDeployResponse

project_deploy_response: ProjectDeployResponse = client.project_deploy(**deploy_params)

In [ ]:
project_deploy_response

In [ ]:
project_deploy_response.id

In [ ]:
access_token: str = client.deployment_token_get(deployment_id=project_deploy_response.id)
access_token

In [ ]:
service_endpoint: str = project_deploy_response.url
service_endpoint

In [ ]:
secret_names: list[str] = ["MLFLOW_REGISTRY_URI", "MLFLOW_TRACKING_TOKEN", "MLFLOW_TRACKING_URI"]
for name in secret_names:
    try:
        client.secret_delete(key=name)
    except:
        pass

client.secret_put(key="MLFLOW_REGISTRY_URI", value=service_endpoint)
client.secret_put(key="MLFLOW_TRACKING_URI", value=service_endpoint)
client.secret_put(key="MLFLOW_TRACKING_TOKEN", value=access_token)

In [ ]:
!ae5 job create --command "GarbageCollection" --schedule "*/10 * * * *" --name "scheduled dev.mlflow.tracking.server garbage collection" "dev.mlflow.tracking.server"
# omit if defined as ae5 secrets:
# --variable MLFLOW_BACKEND_STORE_URI="sqlite:///data/mlflow/dev/store/mydb.sqlite" --variable MLFLOW_TRACKING_GC_TTL="0d0h10m0s"